# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем все необходимые библиотеки для работы

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Откроем и изучим датасет

In [2]:
df = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Посмотрели на датасет ещё раз - предобработка действительно сделана.

## Разбейте данные на выборки

Зададим *test_size = 0.25* и *random_state = 12345*

In [5]:
df_train, df_test_valid = train_test_split(df, test_size=0.25, random_state=12345)

In [6]:
df_test, df_valid = train_test_split(df_test_valid, test_size=0.1, random_state=12345)

Разделим тренировочный датасет на фичи и таргеты

In [7]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

Разделим валидационный датасет на фичи и таргеты

In [8]:
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

Разделим тестовый датасет на фичи и таргеты

In [9]:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

Первым делом изучим модель *Decision Tree* с гиперпараметрами *random_state = 12345* и *max_depth = 1...5*

In [10]:
best_score = 0
scores = []
for depth in range(1, 6):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    if accuracy > best_score:
        best_score = accuracy
        best_model_tree = model_tree
    print(f"Accuracy of {depth} model is:", accuracy)
print("Best score is:", best_score, "for best model tree:", best_model_tree)

Accuracy of 1 model is: 0.7654320987654321
Accuracy of 2 model is: 0.7777777777777778
Accuracy of 3 model is: 0.7777777777777778
Accuracy of 4 model is: 0.7901234567901234
Accuracy of 5 model is: 0.7901234567901234
Best score is: 0.7901234567901234 for best model tree: DecisionTreeClassifier(max_depth=4, random_state=12345)


**Вывод:** по результатам обучения, заметим, что модель решающего дерева показывает лучший *accuracy = 0.7901234567901234*. То есть наилучший результат мы можем достичь при заданных выборках с глубиной дерева равной 3.

Далее изучим работу *Random Forest* с гиперпараметрами *random_state = 4* и *n_estimators=3*

In [12]:
best_score = 0
for i in range(1, 10):
    model_forest = RandomForestClassifier(random_state=4, n_estimators=i)
    model_forest.fit(features_train, target_train)
    predictions = model_forest.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print(f"Accuracy of {i} model is:", accuracy)
    if accuracy > best_score:
        best_score = accuracy
        best_model_forest = model_forest
print("Best score is:", best_score, "for best model forest:", best_model_tree)

Accuracy of 1 model is: 0.7037037037037037
Accuracy of 2 model is: 0.7777777777777778
Accuracy of 3 model is: 0.7283950617283951
Accuracy of 4 model is: 0.7777777777777778
Accuracy of 5 model is: 0.7654320987654321
Accuracy of 6 model is: 0.8148148148148148
Accuracy of 7 model is: 0.8148148148148148
Accuracy of 8 model is: 0.8395061728395061
Accuracy of 9 model is: 0.8148148148148148
Best score is: 0.8395061728395061 for best model forest: DecisionTreeClassifier(max_depth=4, random_state=12345)


**Вывод:** по результатам обучения, заметим, что модель случайного леса показывает лучший *accuracy = 0.8395061728395061*. То есть наилучший результат мы можем достичь при заданных выборках с количеством деревьев равными 4 и 6.

Проверим качество модели LogisticRegression с установленным гиперпараметром *random_state = 12345*

In [13]:
best_score = 0
model_logistic = LogisticRegression(random_state=12345)
model_logistic.fit(features_train, target_train)
predictions = model_logistic.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print(f"Accuracy of our model is:", accuracy)
if accuracy > best_score:
        best_score = accuracy
        best_model_logistic = model_logistic
print("Best score is:", best_score, "for best model logistic:", best_model_tree)

Accuracy of our model is: 0.7283950617283951
Best score is: 0.7283950617283951 for best model logistic: DecisionTreeClassifier(max_depth=4, random_state=12345)


**Вывод:** по результатам обучения, заметим, что модель случайного леса показывает лучший *accuracy = 0.7080745341614907*. То есть наилучший результат мы можем достичь при заданных выборках. 

## Проверьте модель на тестовой выборке

Проверим *accuracy* дерева решений на тестовой выборке

In [14]:
predictions = best_model_tree.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print("Accuracy of our model is:", accuracy)

Accuracy of our model is: 0.7800829875518672


Проверим *accuracy* случайного леса на тестовой выборке

In [15]:
predictions = best_model_forest.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print("Accuracy of our model is:", accuracy)

Accuracy of our model is: 0.7939142461964038


Проверим *accuracy* логистической регресии на тестовой выборке

In [16]:
predictions = best_model_logistic.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print("Accuracy of our model is:", accuracy)

Accuracy of our model is: 0.7012448132780082


**Вывод:** Проверил качество моделей метрикой *accuracy*. Наилучший результат показал случайный лес. Далее шло дерево решений и устпающая моделька двум первым - логистическая регрессия.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
